In [2]:
import torch
import torchvision

print(torch.__version__)
print(torchvision.__version__)

2.2.0.dev20230922+cu121
0.17.0.dev20230925+cu121


In [3]:
import matplotlib.pyplot as plt
import torch 
import torchvision

from torch import nn 
from torchvision import transforms
from torchinfo import summary

from going_modular import data_setup, engine

In [5]:
from pathlib import Path
# Setup device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"

# Setup hyperparameters
NUM_EPOCHS = 5
BATCH_SIZE = 32
HIDDEN_UNITS = 10
LEARNING_RATE = 0.001

# Setup directories
train_dir = Path("data/pizza_steak_sushi/train")
test_dir = Path("data/pizza_steak_sushi/test")

train_dir, test_dir

(WindowsPath('data/pizza_steak_sushi/train'),
 WindowsPath('data/pizza_steak_sushi/test'))